In [1]:
# Uncomment to install dependencies
#!pip install pandas
#!pip install youtube_dl
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
import pandas as pd
import random
import json
import os
from youtube_dl import YoutubeDL

from google_auth_oauthlib.flow import InstalledAppFlow
from apiclient.discovery import build
from apiclient.http import MediaFileUpload
from apiclient import errors

yt_url_prefix = 'https://www.youtube.com/watch?v='
output_folder = 'outputs/'

In [3]:
# GOOGLE DRIVE API CONSTANTS
flow = InstalledAppFlow.from_client_secrets_file('credentials.json', ['https://www.googleapis.com/auth/drive'])
creds = flow.run_local_server(port=0)
folder_id = '1UnDIe4VHMM8bZzIfKEG8NJdQGnNrrxEG'
drive_api = build('drive', 'v3', credentials=creds)

# Helpful function
def get_file_list_from_folder(service, folder_id):
    """Print files belonging to a folder.

    Args:
    service: Drive API service instance.
    folder_id: ID of the folder to print files from.
    """
    kwargs = {
        "q": "'{}' in parents".format(folder_id)
    }
    request = service.files().list(**kwargs)
    while request is not None:
      response = request.execute()
      # Do stuff with response['files']
      request = service.files().list_next(request, response)
    return response

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=786879558582-eh0bho168rp2k1qord3av8r5u10q64sl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50566%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=e2SmXwaXD6L4B8LmrTCgoXG5GXQXu8&access_type=offline


In [4]:
trailers = pd.read_csv("trailers.csv", index_col=0)
#trailers

In [ ]:
ydl_opts = {
    'outtmpl': output_folder + '%(id)s.%(ext)s',
    'format': 'bestvideo'
}

uploaded = get_file_list_from_folder(drive_api, folder_id)
uploaded = set([d['name'] for d in uploaded['files']])


all_to_download = set(trailers['youtube_key'] + ".mp4")
print(f'files already uploaded {uploaded & all_to_download}')
remaining_trailers = all_to_download - uploaded

with YoutubeDL(ydl_opts) as ydl:
    for file_name in remaining_trailers:
        try:
            # Helpful Constants
            yt_link = yt_url_prefix + file_name[:-4]

            # Download Video
            print(f"Downloading {file_name}")
            trailer_yt_info = ydl.extract_info(yt_link, download=True)

            # Upload to Drive
            print(f"Uploading file {file_name}...")
            body = {'name': file_name, 'parents': [folder_id]}
            media = MediaFileUpload(output_folder + file_name)
            fiahl = drive_api.files().create(body=body, media_body=media).execute()
            print(f"Created file '{fiahl.get('name')}' id '{fiahl.get('id')}'.")

            # Delete video from hard drive
            os.remove(output_folder + file_name)
            print(f"Removed {file_name}")
            print("Success!")
        except:
            print("Failed!")
        print()
        


files already uploaded {'uTDVnwwQNTw.mp4', 'z7cEL6n24d4.mp4', 'HddkucqSzSM.mp4', 'K_tLp7T6U1c.mp4', 'uMwECVb1q54.mp4', 'LoVhgaj6PiE.mp4', 'yAQcwKY0Dik.mp4', 'H9Z3_ifFheQ.mp4', '11ptis6I2Zk.mp4', 'qsR8-zxV-3E.mp4', '9vN6DHB6bJc.mp4', 'F-qBD17wwrQ.mp4', 'j8detvC7euw.mp4', 'hEJnMQG9ev8.mp4', 'mfmrPu43DF8.mp4', '6vY9UPiI4eQ.mp4', '95ghQs5AmNk.mp4', 'oYTfYsODWQo.mp4', 'BXUEUwwgIyU.mp4', 'p35JDJLa9ec.mp4', 'bxLAH3M3ywM.mp4', 'iG0P6bjyUNI.mp4', 'XEkeZhWbW7U.mp4', 'iCrUqt9Uf3E.mp4', 'uHBnrJowBZE.mp4', 'EsHlXpbMPOc.mp4', 'XFYWazblaUA.mp4', 'ioUE_5wpg_E.mp4', 'O7zvehDxttM.mp4', 'H0MoWlLPLAM.mp4', 's6SvOIbaulA.mp4', '_IqFJLdV13o.mp4', '34WIbmXkewU.mp4', 'nMft5QDOHek.mp4', 'vizwnpbwO0w.mp4'}
[youtube] 87E6N7ToCxs: Downloading webpage
[youtube] 87E6N7ToCxs: Downloading video info webpage
[download] Destination: outputs/87E6N7ToCxs.mp4
[download] 100% of 28.79MiB in 00:0733MiB/s ETA 00:000
Uploading file 87E6N7ToCxs.mp4...
Created file '87E6N7ToCxs.mp4' id '10l4jCGDvdyBv24_Q_ufomrqT0k00lEBp'.
Remove